In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import numpy as np
from gensim.models import Word2Vec
from nltk.corpus import wordnet
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score

In [9]:
# Clonage du répertoire Git contenant le corpus NFCorpus
#!git clone https://github.com/cr-nlp/project1-2023.git

In [10]:
# Téléchargement des ressources NLTK nécessaires
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialisation du lemmatiseur
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cocog\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cocog\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cocog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cocog\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [11]:


def loadNFCorpus():
    dir = "./project1-2023/"
    
    # Charger les documents
    filename = dir + "dev.docs"
    dicDoc = {}
    with open(filename, encoding='utf-8') as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.split('\t')
        key = tabLine[0]
        value = tabLine[1]
        dicDoc[key] = value

    # Charger les requêtes
    filename = dir + "dev.all.queries"
    dicReq = {}
    with open(filename, encoding='utf-8') as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.split('\t')
        key = tabLine[0]
        value = tabLine[1]
        dicReq[key] = value

    # Charger les scores de pertinence
    filename = dir + "dev.2-1-0.qrel"
    dicReqDoc = defaultdict(dict)
    with open(filename, encoding='utf-8') as file:
        lines = file.readlines()
    for line in lines:
        tabLine = line.strip().split('\t')
        req = tabLine[0]
        doc = tabLine[2]
        score = int(tabLine[3])
        dicReqDoc[req][doc] = score

    return dicDoc, dicReq, dicReqDoc

def text2TokenList(text):
    # Filtrage et lemmatisation des mots
    stopwords_set = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())
    lemmatized_tokens = [
        lemmatizer.lemmatize(word) for word in word_tokens
        if word not in stopwords_set and len(word) > 2
    ]
    return lemmatized_tokens

# Chargement du corpus NFCorpus
dicDoc, dicReq, dicReqDoc = loadNFCorpus()


In [12]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts."""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def advanced_preprocess(text):
    word_tokens = word_tokenize(text.lower())
    filtered_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_tokens if w not in stop_words and len(w) > 2]
    return filtered_words

sample_doc = dicDoc[list(dicDoc.keys())[0]]
preprocessed_sample = advanced_preprocess(sample_doc)
print(preprocessed_sample)


['alkylphenols', 'human', 'milk', 'relation', 'dietary', 'habit', 'central', 'taiwan', 'pubmed', 'ncbi', 'abstract', 'aim', 'study', 'determine', 'concentration', 'num', 'nonylphenol', 'num', 'octylphenol', 'num', 'human', 'milk', 'sample', 'examine', 'related', 'factor', 'include', 'mother', 'demographic', 'dietary', 'habit', 'woman', 'consume', 'median', 'amount', 'cooking', 'oil', 'significantly', 'high', 'concentration', 'num', 'ng/g', 'consume', 'num', 'ng/g', 'num', 'concentration', 'significantly', 'consumption', 'cooking', 'oil', 'beta', 'num', 'num', 'fish', 'oil', 'capsule', 'beta', 'num', 'num', 'adjustment', 'age', 'body', 'mass', 'index', 'bmi', 'concentration', 'significantly', 'consumption', 'fish', 'oil', 'capsule', 'beta', 'num', 'num', 'process', 'fish', 'product', 'beta', 'num', 'num', 'food', 'pattern', 'cooking', 'oil', 'process', 'meat', 'product', 'factor', 'analysis', 'strongly', 'concentration', 'human', 'milk', 'num', 'determination', 'aid', 'suggest', 'food',

In [13]:
from gensim.models import Word2Vec

# Préparation des données pour Word2Vec
corpus_for_word2vec = [advanced_preprocess(doc) for doc in dicDoc.values()]

# Entraînement du modèle Word2Vec
word2vec_model = Word2Vec(sentences=corpus_for_word2vec, vector_size=100, window=5, min_count=1, workers=4)


In [14]:
from rank_bm25 import BM25Okapi

# Préparation du corpus pour BM25
tokenized_corpus = [advanced_preprocess(doc) for doc in dicDoc.values()]
bm25 = BM25Okapi(tokenized_corpus)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Fonction pour créer un vecteur moyen à partir des embeddings d'un texte
def vectorize_text(text, model):
    return np.mean([model.wv[word] for word in text if word in model.wv] or [np.zeros(model.vector_size)], axis=0)

# Exemple de calcul de la similarité pour une requête
query_vector = vectorize_text(advanced_preprocess(dicReq[list(dicReq.keys())[0]]), word2vec_model)
doc_vectors = np.array([vectorize_text(doc, word2vec_model) for doc in tokenized_corpus])

# Calcul de la similarité cosinus pour un échantillon de requête
cosine_similarities = cosine_similarity([query_vector], doc_vectors)


In [ ]:
def combine_scores(bm25_scores, cosine_similarities, alpha=0.5):
    combined_scores = alpha * bm25_scores + (1 - alpha) * cosine_similarities
    return combined_scores

def calculate_true_scores(query_id, dicReqDoc):
    relevant_docs = dicReqDoc[query_id]
    true_scores = [relevant_docs[doc_id] if doc_id in relevant_docs else 0 for doc_id in dicDoc.keys()]
    return true_scores




In [ ]:
sample_query_id = list(dicReq.keys())[0]
sample_query = advanced_preprocess(dicReq[sample_query_id])
bm25_scores = bm25.get_scores(sample_query)

query_vector = vectorize_text(sample_query, word2vec_model)
doc_vectors = np.array([vectorize_text(doc, word2vec_model) for doc in tokenized_corpus])
cosine_similarities = cosine_similarity([query_vector], doc_vectors)[0]

combined_scores = combine_scores(bm25_scores, cosine_similarities, alpha=0.5)

NDCG Score: 1.0


In [ ]:
true_scores = calculate_true_scores(sample_query_id, dicReqDoc)

combined_scores_reduced = combined_scores[:len(true_scores)]

sorted_indices = np.argsort(-combined_scores_reduced)
sorted_true_scores = np.array(true_scores)[sorted_indices]
sorted_combined_scores = combined_scores_reduced[sorted_indices]

True scores length: 5
Combined scores length: 3193
True scores shape: (1, 5)
Combined scores shape: (1, 3193)


In [ ]:
ndcg = ndcg_score([sorted_true_scores], [sorted_combined_scores])
print(f"NDCG Score: {ndcg}")

print("True scores length:", len(true_scores))
print("Combined scores length:", len(combined_scores))
print("True scores shape:", np.array([true_scores]).shape)
print("Combined scores shape:", np.array([combined_scores]).shape)
